In [69]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Model, layers
import tensorflow as tf
import tensorflow.compat.v1 as tf
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
tf.disable_control_flow_v2
tf.compat.v1.disable_eager_execution()

In [50]:
base = pd.read_csv('/content/drive/My Drive/TensorFlow_1.x_guide/autoencoders/CSV/credit_data.csv')
base = base.drop('i#clientid', axis=1)
base = base.dropna()

In [51]:
scaler_x = StandardScaler()
base[['income', 'age', 'loan']] = scaler_x.fit_transform(base[['income', 'age', 'loan']])
X = base.drop('c#default', axis=1)
y = base['c#default']

In [52]:
X.head()

,income,age,loan
0,1.453898,1.336861,1.201907
1,-0.762398,0.536639,0.695744
2,0.836733,1.637207,1.173812
3,-0.183244,0.362998,0.544366
4,1.509532,-1.631534,1.419754


In [53]:
inputs_neurons = 3
hidden_neurons = 2
output_neurons = inputs_neurons

In [54]:
X_placeholder = tf.placeholder(tf.float32, shape=[None, inputs_neurons])

In [55]:
hidden_layer = tf.layers.dense(units = hidden_neurons, inputs = X_placeholder, activation = None)
output_layer = tf.layers.dense(units = output_neurons, inputs = hidden_layer)

In [56]:
error = tf.losses.mean_squared_error(labels= X_placeholder, predictions=output_layer)
optimizer = tf.train.AdadeltaOptimizer(0.01)
train = optimizer.minimize(error)

In [57]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epochs in range(1000):
       cost, _ = sess.run([error, train], feed_dict = {X_placeholder: X})
       if  epochs % 100 == 0:
           print('erro:' + str(cost))
    x2d_encode = sess.run(hidden_layer, feed_dict={X_placeholder: X})
    x3d_decode = sess.run(output_layer, feed_dict={X_placeholder: X})

erro:0.6156245
erro:0.614598
erro:0.6134725
erro:0.6122631
erro:0.6109771
erro:0.60962045
erro:0.60819894
erro:0.6067167
erro:0.6051782
erro:0.6035871


In [58]:
x2d_encode.shape

(1997, 2)

In [59]:
x3d_decode.shape

(1997, 3)

In [60]:
X2 = scaler_x.inverse_transform(X)

In [61]:
X2

array([[6.61559251e+04, 5.90170151e+01, 8.10653213e+03],
       [3.44151540e+04, 4.81171531e+01, 6.56474502e+03],
       [5.73171701e+04, 6.31080495e+01, 8.02095330e+03],
       ...,
       [4.43114493e+04, 2.80171669e+01, 5.52278669e+03],
       [4.37560566e+04, 6.39717958e+01, 1.62272260e+03],
       [6.94365796e+04, 5.61526170e+01, 7.37883360e+03]])

In [62]:
x3d_decode2 = scaler_x.inverse_transform(x3d_decode)

In [63]:
x3d_decode2

array([[5.0707809e+04, 4.6873146e+01, 7.5983755e+03],
       [4.9817645e+04, 4.9643932e+01, 5.8424131e+03],
       [5.2395133e+04, 5.1361565e+01, 7.7439927e+03],
       ...,
       [4.3385402e+04, 3.6135056e+01, 4.1105024e+03],
       [4.8442945e+04, 4.9421814e+01, 4.6014434e+03],
       [4.9129590e+04, 4.3810986e+01, 7.0909614e+03]], dtype=float32)

In [64]:
mae_income = mean_absolute_error(X2[:, 0], x3d_decode2[:,0])

In [65]:
mae_income

12943.305695162695

In [67]:
X_enconde = pd.DataFrame({'attribute1': x2d_encode[:,0], 'attribute2': x2d_encode[:,1], 'class': y})

In [68]:
X_enconde.head()

,attribute1,attribute2,class
0,1.280190,-1.235359,0
1,0.288733,-0.942878,0
2,1.148733,-1.563116,0
3,0.324683,-0.601715,0
4,0.974734,0.595104,1


In [70]:
columns = [tf.feature_column.numeric_column(key=columns) for columns in X_enconde.columns]
X_train, X_test, y_train, y_test = train_test_split(X_enconde, y, test_size = 0.3, random_state=0)

In [72]:
function_train = tf.estimator.inputs.pandas_input_fn(x = X_train, y=y_train, batch_size=8, num_epochs=None, shuffle=True)
classifier = tf.estimator.DNNClassifier(feature_columns=columns, hidden_units=[4,4])
classifier.train(input_fn = function_train, steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp2k1lyqjk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and 

In [73]:
function_test = tf.estimator.inputs.pandas_input_fn(x= X_test, y = y_test, batch_size=8, num_epochs=1000, shuffle=False)
metrics_test = classifier.evaluate(input_fn=function_train, steps=1000)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
The value of AUC returned by this may race with the update so this is deprecated. Please use tf.keras.metrics.AUC instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-01T13:34:38Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp2k1lyqjk/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [500/1000]
INFO:tensorflow:Evaluation [600/1000]
INFO:tensorflow:Evaluation [700/1000]
INFO:tensorflow:Evaluation [800/1000]
INFO:tensorflow:Evaluation [900/1000]
INFO:tensorflow:Evaluation [1000/1000]
INFO:tensorflow:Inference Time : 2.51136s
INFO:tensorflow:Finished evaluation at 2020-07-01-13:34:41
INFO:tensorflow:Saving dict for global s

In [74]:
metrics_test

{'accuracy': 1.0,
 'accuracy_baseline': 0.85487497,
 'auc': 1.0,
 'auc_precision_recall': 1.0,
 'average_loss': 0.000984362,
 'global_step': 1000,
 'label/mean': 0.145125,
 'loss': 0.007874896,
 'precision': 1.0,
 'prediction/mean': 0.14554928,
 'recall': 1.0}